In [ ]:
# Importing required libraries
import pandas as pd
import numpy as np
import os
from glob import glob
from PIL import Image
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.utils.class_weight import compute_class_weight
from keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50, MobileNet, DenseNet121
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Conv2D, MaxPooling2D, Input, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.backend as K
import tensorflow as tf

In [ ]:
# Suppress warnings
warnings.filterwarnings('ignore')


In [ ]:
# Read metadata
skinDf = pd.read_csv('/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')

# Set image size (matching paper specification: 120x120)
img_size = (120, 120)

In [ ]:
# ============================================
# FOCAL LOSS IMPLEMENTATION
# ============================================
# Focal Loss helps the model focus on hard-to-classify examples (minority classes)
def focal_loss(gamma=2., alpha=0.25):
    """
    Focal loss for multi-classification
    FL(p_t) = -alpha * (1 - p_t)^gamma * log(p_t)
    
    gamma: focusing parameter (default=2)
    alpha: balancing parameter (default=0.25)
    """
    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    return focal_loss_fixed

# ============================================
# WEIGHTED CATEGORICAL CROSS-ENTROPY
# ============================================
def weighted_categorical_crossentropy(weights):
    """
    Weighted cross-entropy loss that gives more importance to minority classes
    weights: array of weights for each class
    """
    weights = K.variable(weights)
    def loss(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    return loss

print("Loss functions loaded successfully!")

In [ ]:
# Encode labels
labelEncoder = LabelEncoder()
skinDf['label'] = labelEncoder.fit_transform(skinDf['dx'])

# Print original class distribution
print("Original Class Distribution:")
print(skinDf['dx'].value_counts())
print("\nClass label mapping:")
for i, class_name in enumerate(labelEncoder.classes_):
    print(f"Class {i}: {class_name} ({len(skinDf[skinDf['label'] == i])} images)")

# ============================================
# STRATEGY 1: Intelligent Resampling
# ============================================
# Instead of balancing to same count, we'll use a hybrid approach:
# - Oversample minority classes with augmentation
# - Keep majority classes at reasonable levels

# Define target samples per class (based on sqrt of max class size for better balance)
max_samples = skinDf['label'].value_counts().max()
target_samples = int(np.sqrt(max_samples) * 50)  # ~4000 samples per class

print(f"\nTarget samples per class: {target_samples}")

dfs_by_label_resampled = {}
for label in range(7):
    df_label = skinDf[skinDf['label'] == label]
    current_count = len(df_label)
    
    if current_count < target_samples:
        # Oversample minority classes
        df_label_resampled = resample(df_label, 
                                       n_samples=target_samples, 
                                       replace=True, 
                                       random_state=42)
    else:
        # Keep majority class but don't reduce too much
        df_label_resampled = resample(df_label, 
                                       n_samples=min(current_count, target_samples), 
                                       replace=False, 
                                       random_state=42)
    
    dfs_by_label_resampled[label] = df_label_resampled
    print(f"Class {label}: {current_count} -> {len(df_label_resampled)} images")

balanced_df = pd.concat(dfs_by_label_resampled.values()).sample(frac=1, random_state=42).reset_index(drop=True)

print(f"\nTotal balanced dataset size: {len(balanced_df)}")
print(f"Expected training size (60%): {int(len(balanced_df) * 0.6)}")

In [ ]:
# Load images
imgPath = {os.path.splitext(os.path.basename(x))[0]: x for x in glob(os.path.join('/kaggle/input/skin-cancer-mnist-ham10000/', '*', '*.jpg'))}
balanced_df['image'] = balanced_df['image_id'].map(imgPath.get).map(lambda x: np.asarray(Image.open(x).resize(img_size)) / 255)

# Prepare features and labels
x = np.asarray(balanced_df['image'].to_list())
y = to_categorical(balanced_df['label'], num_classes=7)

# Split data: 60% train, 20% validation, 20% test (as per paper section 3.2)
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.4, random_state=42, shuffle=True, stratify=y)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42, shuffle=True, stratify=y_temp)

print(f"Training set: {x_train.shape}")
print(f"Validation set: {x_val.shape}")
print(f"Test set: {x_test.shape}")

# ============================================
# STRATEGY 2: Calculate Class Weights
# ============================================
# Calculate class weights to give more importance to minority classes in loss function
y_train_labels = np.argmax(y_train, axis=1)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_labels),
    y=y_train_labels
)

# Convert to dictionary format for Keras
class_weight_dict = dict(enumerate(class_weights))

print("\nClass Weights (higher = more important):")
for i, weight in class_weight_dict.items():
    class_name = labelEncoder.classes_[i]
    print(f"Class {i} ({class_name}): {weight:.3f}")

In [ ]:
# ============================================
# STRATEGY 3: Data Augmentation
# ============================================
# Apply aggressive augmentation to training data to improve generalization
# This is especially important for minority classes

train_datagen = ImageDataGenerator(
    rotation_range=40,           # Randomly rotate images by up to 40 degrees
    width_shift_range=0.2,       # Randomly shift images horizontally (20%)
    height_shift_range=0.2,      # Randomly shift images vertically (20%)
    shear_range=0.2,             # Shear transformation
    zoom_range=0.2,              # Random zoom
    horizontal_flip=True,        # Randomly flip images horizontally
    vertical_flip=True,          # Randomly flip images vertically
    brightness_range=[0.8, 1.2], # Adjust brightness
    fill_mode='nearest'          # Fill newly created pixels
)

# Validation and test data should NOT be augmented
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# Create generators
batch_size = 32  # Increased from 8 for better training stability

train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size, shuffle=True)
val_generator = val_datagen.flow(x_val, y_val, batch_size=batch_size, shuffle=False)

print("Data augmentation configured successfully!")
print(f"Batch size: {batch_size}")
print(f"Steps per epoch: {len(x_train) // batch_size}")

# ============================================
# MODEL COMPILATION WITH IMPROVED LOSS FUNCTION
# ============================================
# Options: 'categorical_crossentropy', 'focal_loss', 'weighted_crossentropy'
LOSS_FUNCTION = 'focal_loss'  # Change this to experiment

if LOSS_FUNCTION == 'focal_loss':
    loss = focal_loss(gamma=2.0, alpha=0.25)
    print("Using Focal Loss (focuses on hard-to-classify examples)")
elif LOSS_FUNCTION == 'weighted_crossentropy':
    loss = weighted_categorical_crossentropy(class_weights)
    print("Using Weighted Categorical Cross-Entropy")
else:
    loss = 'categorical_crossentropy'
    print("Using standard Categorical Cross-Entropy")

# Compile model with optimized learning rate
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss=loss,
    metrics=['accuracy']
)

print(f"\nModel compiled with {LOSS_FUNCTION}")

In [ ]:
# ============================================
# TRAINING WITH ALL IMPROVEMENTS
# ============================================

# Callbacks for better training
callbacks = [
    # Stop training if validation loss doesn't improve for 10 epochs
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Reduce learning rate when validation loss plateaus
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=0.00001,
        verbose=1
    ),
    
    # Save best model
    ModelCheckpoint(
        'best_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# Train model with data augmentation and class weights
print("\n" + "="*50)
print("TRAINING MODEL WITH ALL IMPROVEMENTS")
print("="*50)
print("✓ Data Augmentation: Enabled")
print("✓ Class Weights: Enabled")
print(f"✓ Loss Function: {LOSS_FUNCTION}")
print("✓ Early Stopping: Enabled")
print("✓ Learning Rate Reduction: Enabled")
print("="*50 + "\n")

history = model.fit(
    train_generator,
    steps_per_epoch=len(x_train) // batch_size,
    epochs=200,
    validation_data=val_generator,
    validation_steps=len(x_val) // batch_size,
    callbacks=callbacks,
    class_weight=class_weight_dict,  # Apply class weights
    verbose=1
)

print("\nTraining completed!")

In [ ]:
# ============================================
# MODEL EVALUATION WITH DETAILED METRICS
# ============================================

# Evaluate on test set
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)

print("\n" + "="*50)
print("TEST SET RESULTS")
print("="*50)
print(f'Overall Test Accuracy: {test_accuracy*100:.2f}%')
print(f'Overall Test Loss: {test_loss:.4f}')
print("="*50)

# Get predictions
y_pred = model.predict(x_test, verbose=0)
y_true_int = np.argmax(y_test, axis=1)
y_pred_int = np.argmax(y_pred, axis=1)

# ============================================
# PER-CLASS ACCURACY (Most Important!)
# ============================================
print("\n" + "="*50)
print("PER-CLASS ACCURACY")
print("="*50)

class_accuracies = []
for i in range(7):
    class_name = labelEncoder.classes_[i]
    # Find indices where true label is this class
    class_indices = np.where(y_true_int == i)[0]
    if len(class_indices) > 0:
        # Calculate accuracy for this class
        class_correct = np.sum(y_pred_int[class_indices] == i)
        class_total = len(class_indices)
        class_acc = (class_correct / class_total) * 100
        class_accuracies.append(class_acc)
        
        status = "✓" if class_acc >= 90 else "✗" if class_acc < 80 else "~"
        print(f"{status} Class {i} ({class_name:30s}): {class_acc:6.2f}% ({class_correct}/{class_total})")
    else:
        class_accuracies.append(0)
        print(f"✗ Class {i} ({class_name:30s}): No test samples")

print("="*50)
print(f"Average Per-Class Accuracy: {np.mean(class_accuracies):.2f}%")
print(f"Minimum Class Accuracy: {np.min(class_accuracies):.2f}%")
print(f"Maximum Class Accuracy: {np.max(class_accuracies):.2f}%")
print("="*50)

# ============================================
# DETAILED CLASSIFICATION REPORT
# ============================================
print("\n" + "="*50)
print("DETAILED CLASSIFICATION REPORT")
print("="*50)
print(classification_report(y_true_int, y_pred_int, 
                          target_names=labelEncoder.classes_,
                          digits=4))

# ============================================
# CONFUSION MATRIX
# ============================================
cm = confusion_matrix(y_true_int, y_pred_int)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=labelEncoder.classes_,
            yticklabels=labelEncoder.classes_)
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('True', fontsize=12)
plt.title('Confusion Matrix - Skin Cancer Classification', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# ============================================
# TRAINING HISTORY VISUALIZATION
# ============================================
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot accuracy
axes[0].plot(history.history['accuracy'], label='Training Accuracy')
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy')
axes[0].set_title('Model Accuracy Over Time', fontsize=14)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot loss
axes[1].plot(history.history['loss'], label='Training Loss')
axes[1].plot(history.history['val_loss'], label='Validation Loss')
axes[1].set_title('Model Loss Over Time', fontsize=14)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# ============================================
# EXPORT RESULTS FOR PAPER
# ============================================
print("\n" + "="*50)
print("EXPORTING RESULTS FOR PAPER")
print("="*50)

# Create output directory
import os
os.makedirs('paper_results', exist_ok=True)

# 1. Export Per-Class Accuracy to CSV
class_results = []
for i in range(7):
    class_name = labelEncoder.classes_[i]
    class_indices = np.where(y_true_int == i)[0]
    if len(class_indices) > 0:
        class_correct = np.sum(y_pred_int[class_indices] == i)
        class_total = len(class_indices)
        class_acc = (class_correct / class_total) * 100
        
        # Calculate precision, recall, f1 for this class
        tp = class_correct
        fp = np.sum((y_pred_int == i) & (y_true_int != i))
        fn = class_total - class_correct
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        class_results.append({
            'Class': i,
            'Disease': class_name,
            'True_Positives': tp,
            'False_Positives': fp,
            'False_Negatives': fn,
            'Total_Samples': class_total,
            'Accuracy': class_acc,
            'Precision': precision * 100,
            'Recall': recall * 100,
            'F1_Score': f1 * 100
        })

df_class_results = pd.DataFrame(class_results)
df_class_results.to_csv('paper_results/per_class_metrics.csv', index=False)
print("✓ Saved: paper_results/per_class_metrics.csv")

# 2. Export Overall Model Performance
overall_results = {
    'Model_Type': [MODEL_TYPE],
    'Loss_Function': [LOSS_FUNCTION],
    'Overall_Test_Accuracy': [test_accuracy * 100],
    'Overall_Test_Loss': [test_loss],
    'Average_Per_Class_Accuracy': [np.mean([r['Accuracy'] for r in class_results])],
    'Min_Class_Accuracy': [np.min([r['Accuracy'] for r in class_results])],
    'Max_Class_Accuracy': [np.max([r['Accuracy'] for r in class_results])],
    'Training_Samples': [len(x_train)],
    'Validation_Samples': [len(x_val)],
    'Test_Samples': [len(x_test)],
    'Data_Augmentation': ['Yes'],
    'Class_Weights': ['Yes'],
    'Batch_Size': [batch_size]
}
df_overall = pd.DataFrame(overall_results)
df_overall.to_csv('paper_results/overall_model_performance.csv', index=False)
print("✓ Saved: paper_results/overall_model_performance.csv")

# 3. Export Confusion Matrix to CSV
df_cm = pd.DataFrame(cm, 
                     index=[f"{i}_{name}" for i, name in enumerate(labelEncoder.classes_)],
                     columns=[f"{i}_{name}" for i, name in enumerate(labelEncoder.classes_)])
df_cm.to_csv('paper_results/confusion_matrix.csv')
print("✓ Saved: paper_results/confusion_matrix.csv")

# 4. Export Training History
df_history = pd.DataFrame({
    'Epoch': range(1, len(history.history['accuracy']) + 1),
    'Training_Accuracy': history.history['accuracy'],
    'Validation_Accuracy': history.history['val_accuracy'],
    'Training_Loss': history.history['loss'],
    'Validation_Loss': history.history['val_loss']
})
df_history.to_csv('paper_results/training_history.csv', index=False)
print("✓ Saved: paper_results/training_history.csv")

# 5. Save Confusion Matrix as Image
plt.figure(figsize=(14, 12))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=labelEncoder.classes_,
            yticklabels=labelEncoder.classes_,
            cbar_kws={'label': 'Count'})
plt.xlabel('Predicted Class', fontsize=14, fontweight='bold')
plt.ylabel('True Class', fontsize=14, fontweight='bold')
plt.title(f'Confusion Matrix - {MODEL_TYPE.upper()} Model\nOverall Accuracy: {test_accuracy*100:.2f}%', 
          fontsize=16, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('paper_results/confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.savefig('paper_results/confusion_matrix.pdf', bbox_inches='tight')
print("✓ Saved: paper_results/confusion_matrix.png")
print("✓ Saved: paper_results/confusion_matrix.pdf")
plt.close()

# 6. Save Training History Plot
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Accuracy plot
axes[0].plot(history.history['accuracy'], label='Training', linewidth=2, marker='o', markersize=3)
axes[0].plot(history.history['val_accuracy'], label='Validation', linewidth=2, marker='s', markersize=3)
axes[0].set_title('Model Accuracy Over Time', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Loss plot
axes[1].plot(history.history['loss'], label='Training', linewidth=2, marker='o', markersize=3)
axes[1].plot(history.history['val_loss'], label='Validation', linewidth=2, marker='s', markersize=3)
axes[1].set_title('Model Loss Over Time', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('paper_results/training_history.png', dpi=300, bbox_inches='tight')
plt.savefig('paper_results/training_history.pdf', bbox_inches='tight')
print("✓ Saved: paper_results/training_history.png")
print("✓ Saved: paper_results/training_history.pdf")
plt.close()

# 7. Save Per-Class Accuracy Bar Chart
plt.figure(figsize=(14, 8))
class_names_short = [name[:20] for name in labelEncoder.classes_]
accuracies = [r['Accuracy'] for r in class_results]
colors = ['green' if acc >= 90 else 'orange' if acc >= 80 else 'red' for acc in accuracies]

bars = plt.bar(range(7), accuracies, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
plt.axhline(y=90, color='green', linestyle='--', linewidth=2, label='90% Target', alpha=0.5)
plt.axhline(y=80, color='orange', linestyle='--', linewidth=2, label='80% Threshold', alpha=0.5)

# Add value labels on bars
for i, (bar, acc) in enumerate(zip(bars, accuracies)):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{acc:.1f}%', ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.xlabel('Disease Class', fontsize=14, fontweight='bold')
plt.ylabel('Accuracy (%)', fontsize=14, fontweight='bold')
plt.title(f'Per-Class Accuracy - {MODEL_TYPE.upper()} Model', fontsize=16, fontweight='bold')
plt.xticks(range(7), class_names_short, rotation=45, ha='right', fontsize=10)
plt.ylim(0, 105)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig('paper_results/per_class_accuracy.png', dpi=300, bbox_inches='tight')
plt.savefig('paper_results/per_class_accuracy.pdf', bbox_inches='tight')
print("✓ Saved: paper_results/per_class_accuracy.png")
print("✓ Saved: paper_results/per_class_accuracy.pdf")
plt.close()

# 8. Create Summary Report
summary_text = f"""
SKIN CANCER CLASSIFICATION - MODEL PERFORMANCE REPORT
{'='*70}

MODEL CONFIGURATION:
  - Model Type: {MODEL_TYPE.upper()}
  - Loss Function: {LOSS_FUNCTION}
  - Data Augmentation: Yes
  - Class Weights: Yes
  - Batch Size: {batch_size}
  
DATASET SPLIT:
  - Training Samples: {len(x_train)}
  - Validation Samples: {len(x_val)}
  - Test Samples: {len(x_test)}

OVERALL PERFORMANCE:
  - Overall Test Accuracy: {test_accuracy*100:.2f}%
  - Overall Test Loss: {test_loss:.4f}
  - Average Per-Class Accuracy: {np.mean([r['Accuracy'] for r in class_results]):.2f}%
  - Minimum Class Accuracy: {np.min([r['Accuracy'] for r in class_results]):.2f}%
  - Maximum Class Accuracy: {np.max([r['Accuracy'] for r in class_results]):.2f}%

PER-CLASS PERFORMANCE:
{'='*70}
"""

for result in class_results:
    summary_text += f"""
Class {result['Class']}: {result['Disease']}
  - Accuracy: {result['Accuracy']:.2f}%
  - Precision: {result['Precision']:.2f}%
  - Recall: {result['Recall']:.2f}%
  - F1-Score: {result['F1_Score']:.2f}%
  - Test Samples: {result['Total_Samples']}
"""

summary_text += f"\n{'='*70}\n"

with open('paper_results/model_summary_report.txt', 'w') as f:
    f.write(summary_text)
print("✓ Saved: paper_results/model_summary_report.txt")

print("\n" + "="*50)
print("ALL RESULTS EXPORTED SUCCESSFULLY!")
print("Check the 'paper_results' folder for:")
print("  - CSV files (metrics, confusion matrix, training history)")
print("  - PNG images (high resolution, for presentations)")
print("  - PDF files (vector format, for papers)")
print("  - TXT summary report")
print("="*50)

In [ ]:
# Reduce learning rate when a metric has stopped improving
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Train model (using validation set for validation, not test set)
history = model.fit(x_train, y_train, epochs=200, batch_size=8, validation_data=(x_val, y_val), callbacks=[EarlyStopping(patience=5), reduce_lr])

In [ ]:
# Predict on test set
y_pred = model.predict(x_test)

# Convert one-hot encoded labels to integer labels
y_true_int = np.argmax(y_test, axis=1)
y_pred_int = np.argmax(y_pred, axis=1)

# Generate confusion matrix
cm = confusion_matrix(y_true_int, y_pred_int)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4', 'Class 5', 'Class 6'],
            yticklabels=['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4', 'Class 5', 'Class 6'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import numpy as np
y_pred = model.predict(x_test)
import seaborn as sns

# Convert one-hot encoded labels to integer labels
y_true_int = np.argmax(y_test, axis=1)
y_pred_int = np.argmax(y_pred, axis=1)

# Generate confusion matrix
cm = confusion_matrix(y_true_int, y_pred_int)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4', 'Class 5', 'Class 6'],
            yticklabels=['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4', 'Class 5', 'Class 6'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()
